#### Feedback Control

In [21]:
import matplotlib
%matplotlib inline
import pandas as pd
np = pd.np
deg2rad = np.pi / 180.

#### Sensor Measurement Error

In [22]:
def compass_measurement(true_heading, std=3*deg2rad, bias=2*deg2rad):
    return (std * deg2rad * random.randn() + bias) % (2 * np.pi)



#### A Controller (Autopilot, Servo Control, Linear Feedback Control Law)

In [23]:
class Autopilot(object):
    
    def __init__(self, gain=1, damping=.1, integrator=0.01, dt=1):
        self.previous_heading_error = 0
        self.heading_error_sum = 0
        
        self.dt = dt                  # T_s
        self.gain = gain              # K_p
        self.integrator = integrator  # K_i
        self.damping = damping        # K_d
    
    def compute_rudder_angle(self, heading_error, dt=None):
        self.dt = dt or self.dt
        heading_error_rate = (heading_error - self.previous_heading_error) / self.dt
 
        self.previous_heading_error = heading_error
        self.heading_error_sum += heading_error * self.dt
        
        rudder_angle = - (self.gain * heading_error +
                          self.damping * heading_error_rate +
                          self.integrator * self.heading_error_sum
                          )
        return min(max(rudder_angle, -45 * deg2rad), +45 * deg2rad)
    
    def __call__(self, *args, **kwargs):
        return self.compute_rudder_angle(*args, **kwargs)

#### Actuator Model (ship's rudder)

In [24]:
def rudder_angle_turning_rate(rudder_angle):
    return min(max(0.1 * rudder_angle - 0.01 * rudder_angle ** 2, -5 * deg2rad), + 5 * deg2rad)


### Simulation
#### Initial Conditions

In [ ]:
t = 0  # time
position = np.array([0., 0.]) # La Cruz Mexico = 23° 55' N / 106° 54' W
desired_heading = 225 * deg2rad  # SW
actual_heading = 0 * deg2rad
speed = 100  # 2 meter/second = 120 meters/minute = 7.2 km/h = ~= 4.5 miles/h
rudder_angle = 0  # + = starboard (right), - = port (left) 

#### Simulation Configuration Parameters


In [27]:
dt = 60  # 60 seconds between "samples"
total_samples = 30 * 24 * 60  # 30 days = ? minutes from Mexico to the Marquesas
total_time = total_samples * 60  # time in seconds... always :p

#### Initialize the Autopilot and Ship's Log

In [28]:
autopilot = Autopilot(gain=1, damping=.1, integrator=0.01)
ships_log = []

#### Run the Simulation

In [29]:
for i in range(total_samples):
    t += dt
    rudder_angle = autopilot.compute_rudder_angle(actual_heading - desired_heading) 
    actual_heading += dt * rudder_angle_turning_rate(rudder_angle)
    position += dt * speed * np.array([np.sin(actual_heading), np.cos(actual_heading)])
    # can you guess the origin of the word "log" as in "log file", "log book",  or "ship's log"?
    ships_log += [[t, position[0], position[1], desired_heading, actual_heading, rudder_angle]]
    # print(ships_log[-1])

#### Review the Log

In [30]:
df = pd.DataFrame(ships_log, columns=['time', 'x', 'y', 'desired_heading', 'actual_heading', 'rudder'])
df

,time,x,y,desired_heading,actual_heading,rudder
0,2592060,47415487.119126,6.390646e+07,3.926991,7.107739,0.785398
1,2592120,47420878.150902,6.390383e+07,3.926991,2.025240,-0.785398
2,2592180,47421383.553432,6.390980e+07,3.926991,6.367519,0.785398
3,2592240,47427140.211734,6.391150e+07,3.926991,1.285020,-0.785398
4,2592300,47423481.038801,6.391625e+07,3.926991,5.627299,0.785398
5,2592360,47426590.518651,6.392138e+07,3.926991,0.544800,-0.785398
6,2592420,47420681.835185,6.392243e+07,3.926991,4.887078,0.785398
7,2592480,47419516.759053,6.392831e+07,3.926991,-0.195421,-0.785398
8,2592540,47414450.933929,6.392510e+07,3.926991,4.146858,0.785398
9,2592600,47416130.440596,6.393086e+07,3.926991,0.283708,-0.607012


#### What's Wrong?

* Did we make it to the South Pacific?  
* What's wrong with the position?  
* What's wrong with the turning rate and rudder angle?  
* What can we do to fix it?  